Imports

In [1]:
import f90nml
import numpy as np
from pint import UnitRegistry; AssignQuantity = UnitRegistry().Quantity
from QLCstuff2 import getNQLL
import os
import reference_solution as refsol
from scipy.fft import rfft
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import icepinn as ip

torch.set_default_dtype(torch.float64)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

device = ip.get_device()

cuda
cuda


In [2]:
# Read in GI parameters
inputfile = "GI parameters - Reference limit cycle (for testing).nml"
GI=f90nml.read(inputfile)['GI']
nx_crystal = GI['nx_crystal']
L = GI['L']
NBAR = GI['Nbar']
NSTAR = GI['Nstar']

# Define t range
RUNTIME = 5
NUM_T_STEPS = RUNTIME + 1

# Define initial conditions
Ntot_init_1D = np.ones(nx_crystal)
Nqll_init_1D = getNQLL(Ntot_init_1D,NSTAR,NBAR)

# Define x, t pairs for training
X_QLC = np.linspace(-L,L,nx_crystal)
t_points = np.linspace(0, RUNTIME, NUM_T_STEPS)
x, t = np.meshgrid(X_QLC, t_points)
training_set = torch.tensor(np.column_stack((x.flatten(), t.flatten()))).to(device)

In [3]:
# Define model attributes; instantiate model
model_dimensions = torch.tensor([8, 80]).to(device)
is_sf_PINN = torch.tensor(True)
model = ip.IcePINN(
	num_hidden_layers=model_dimensions[0], 
	hidden_layer_size=model_dimensions[0],
	is_sf_PINN=is_sf_PINN.item()).to(device)

# Attach model attributes as buffers so they can be saved and loaded
model.register_buffer('dimensions', model_dimensions)
model.register_buffer('is_sf_PINN', is_sf_PINN)

# Initialize model weights with HE initialization
def init_HE(m):
		if type(m) == nn.Linear:
			nn.init.kaiming_normal_(m.weight)
model.apply(init_HE)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

Train model

In [4]:
MODEL_NAME = "TestPINN"
PATH = './models/'+MODEL_NAME

ip.train_IcePINN(
    model, 
    optimizer, 
    training_set, 
    epochs=50_000, 
    save_path=PATH, 
    print_every=1_000, 
    print_gradients=False)

Commencing PINN training for 50000 epochs.
Loss at epoch [1000/50000]: Ntot = 224950.939, Nqll = 183995.374.
Loss at epoch [2000/50000]: Ntot = 211704.206, Nqll = 136898.588.
Loss at epoch [3000/50000]: Ntot = 465191.361, Nqll = 413599.117.
Loss at epoch [4000/50000]: Ntot = 867276.632, Nqll = 676487.454.
Loss at epoch [5000/50000]: Ntot = 293638.566, Nqll = 118230.419.
Loss at epoch [6000/50000]: Ntot = 805975.467, Nqll = 668814.447.
Loss at epoch [7000/50000]: Ntot = 41798.692, Nqll = 14887.578.
Loss at epoch [8000/50000]: Ntot = 4635.391, Nqll = 106.896.
Loss at epoch [9000/50000]: Ntot = 1127.295, Nqll = 32.371.
Loss at epoch [10000/50000]: Ntot = 1397.694, Nqll = 45.347.
Loss at epoch [11000/50000]: Ntot = 10089.684, Nqll = 976.601.
Loss at epoch [12000/50000]: Ntot = 32960.282, Nqll = 3813.445.
Loss at epoch [13000/50000]: Ntot = 120984.805, Nqll = 6034.735.
Loss at epoch [14000/50000]: Ntot = 1703738.489, Nqll = 428707.893.
Loss at epoch [15000/50000]: Ntot = 431402.147, Nqll = 